In [ ]:
percentiles = (0.000001, 0.95)
bounds = {f: (data[f].quantile(percentiles[0]), data[f].quantile(percentiles[1])) for f in X.columns}

rng = np.random.default_rng(42)
N = 50_000
candidates = {f: rng.uniform(low=b[0], high=b[1], size=N) for f, b in bounds.items()}
Xcand = pd.DataFrame(candidates)[X.columns]
ycand = model.predict(Xcand) #CHANGE "model" TO YOUR INDIVIDUAL MODEL

imin = int(np.argmin(ycand))
best_combo = Xcand.iloc[imin].to_dict()
best_pred  = ycand[imin]

print("\n=== SVR-suggested first-brake setup (within observed range) ===")
for k, v in best_combo.items():
    print(f"{k}: {v:,.4f}")
print(f"Predicted Target_CURRENTLAPTIMEINMS: {best_pred:,.3f}")

In [ ]:
percentiles = (0.05, 0.95)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)
y_scaled_df = pd.DataFrame(y_scaled, columns=["Target_CURRENTLAPTIMEINMS"])

bounds = {
    f: (X_scaled_df[f].quantile(percentiles[0]), X_scaled_df[f].quantile(percentiles[1]))
    for f in X_scaled_df.columns
}

rng = np.random.default_rng(42)
N = 50_000

candidates = {
    f: rng.uniform(low=b[0], high=b[1], size=N)
    for f, b in bounds.items()
}
Xcand = pd.DataFrame(candidates)[X_scaled_df.columns]

ycand = model.predict(Xcand) #CHANGE "model" TO YOUR INDIVIDUAL MODEL

ycand_unscaled = scaler_y.inverse_transform(ycand.reshape(-1, 1)).ravel()

imin = int(np.argmin(ycand_unscaled))
best_combo_scaled = Xcand.iloc[imin].to_frame().T
best_combo_unscaled = pd.DataFrame(
    scaler_X.inverse_transform(best_combo_scaled),
    columns=X.columns
)

best_pred = ycand_unscaled[imin]

print("\n=== XGBoost-suggested first-brake setup (within observed range) ===")
for k, v in best_combo_unscaled.iloc[0].items():
    print(f"{k}: {v:,.4f}")
print(f"Predicted Target_CURRENTLAPTIMEINMS: {best_pred:,.3f}")